# Предобработка данных
В файле prepare_dataset.py производиться подготовка меток, выделение валидационной выборки и аугментация тренировочных изображений. По умолчанию при запуске файла аугментация не производится, поскольку на это уходит несколько часов. Вы можете включить её, добавив параметр --augment_data 1.

In [ ]:
!python ./prepare_dataset.py --data_dir ./participants 

Выбрав модель, я обучил её на всём доступном датасете. Также для уменьшения числа ложноположительных ответов я дополнил его изображениями без меток, которые доступны в директории ./background_images. Они взяты из общедоступного набора данных Hotels-50K, с которым я раньше работал.  
В итоговом тренировочном наборе было 8216 изображений (2002 исходных + 5514 аугментированных + 700 без меток)

# Обучение
Я использовал предобученную модель архитектуры YOLOv7. Ссылку на скачивание весов предобученной модели можно найти в [репозитории YOLOv7](https://github.com/WongKinYiu/yolov7), или можете просто скачать по [ссылке](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6e_training.pt) (318 Мб). Для запуска обучения скачайте её и поместите в каталог с проектом  

Веса лучшей обученной мной модели можно скачать с моего Google диска по [ссылке](https://drive.google.com/u/0/uc?id=156ID04ILe_MkjvHTFS21SBcauvsrIY06&export=download) (1.2 Гб). Для получения предсказания скачайте модель и также поместите её в каталог с проектом  

Результаты обучения (показатели метрик по каждой эпохе) находятся в файле result.txt. За 64 часа на RTX 3090 было пройдено 153 эпохи обучения.  
Все команды я запускал из терминала Jupyter Notebook, но можно и в текущем блокноте

In [ ]:
!python ./yolov7/train_aux.py --workers 8 --device 0 --batch-size 4 --data ./data_for_yolo/dataset.yaml --img 1280 1280 --epochs 300 --cfg ./yolov7/cfg/training/yolov7-e6e.yaml --weights 'yolov7-e6e_training.pt' --name yolov7-e6e-custom

# Обнаружение
Если файл detect запускается в Google Colab, можеть возникнуть ошибка в файле yolov7/utils/google_utils.py. Для её обхода достаточно закоментировать строку 31 файла yolov7/utils/google_utils.py:  
```tag = subprocess.check_output('git tag', shell=True).decode().split()[-1]```

In [ ]:
!python ./yolov7/detect.py --weights ./yolov7.pt --device 0 --source ./participants/test/images --img-size 1280 --save-txt --save-conf

In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random


def get_soliton_labels_df_simple_solution(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
            
            
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = int(val[0]), val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
#             if cls == 1 or (cls == 2 and conf > 0.51) or (cls == 3 and conf > 0.51) or (cls == 0 and conf > 0.6) or (cls == 4 and conf > 0.6):
#             if (cls == 1 and conf > 0.29) or (cls == 2 and conf > 0.0) or (cls == 3 and conf > 0.0) or (cls == 0 and conf > 0.0) or (cls == 4 and conf > 0.0):
            simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
    return simple_solution

In [ ]:
simple_solution = get_soliton_labels_df_simple_solution('runs/detect/exp/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [ ]:
simple_solution.to_csv("sample_solution.csv", sep=';', index=False)